In [21]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [22]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [23]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [24]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

# create the list of index of scenarios out of sample
out_of_sample = []
for i in 1:n_scenarios
    if i ∉ selected_scenarios
        push!(out_of_sample, i)
    end
end

unselected_scenarios = Dict()
counter = 1
for i in out_of_sample
    unselected_scenarios[counter] = all_scenarios[i]
    counter += 1
end

alpha = 0.9
beta = 0.8
objectiv_values = []
exp_profits = []
profits_in = []
CVaR_values = []


Any[]

In [25]:
using CSV, Tables

CSV.write("in_sample_scenarios.csv",  Tables.table(selected_scenarios), writeheader=false)

ArgumentError: ArgumentError: Package Tables not found in current path, maybe you meant `import/using .Tables`.
- Otherwise, run `import Pkg; Pkg.add("Tables")` to install the Tables package.

In [26]:

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)
unregister(model, :p_DA)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, delta_up[1:W,1:hours])
@variable(model, delta_down[1:W,1:hours])
@variable(model, zeta)
@variable(model, eta[1:W] >= 0)

# Define the objective function
@objective(model, Max, (1-beta) * sum(1/W *
(scenarios[i][hour,"price"] * p_DA[hour]
+ scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)
+ beta  * (zeta - (1/(1-alpha)) * sum(1/W * eta[i] for i in 1:W)))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
#@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)
@constraint(model, [i in 1:W], eta[i] >= 0)
@constraint(model, [i in 1:W], -1 * sum((scenarios[i][hour,"price"] * p_DA[hour]
+ scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours) + zeta - eta[i] <= 0)



# Solve the optimization problem
optimize!(model)

# save the p_DA values
p_DA_values = value.(p_DA)

exp_profit = sum(1/W .*
(scenarios[i][hour,"price"] * value.(p_DA[hour])
+ scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)

# save the profit of each scenario
for i in 1:W
    push!(profits_in, sum((scenarios[i][hour,"price"] * value.(p_DA[hour])
    + scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours))
end


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    println("Expected profit ", exp_profit)
    println("p_DA: ", value.(p_DA))
    
else
    println("No optimal solution found")
end


Optimal solution found
Expected profit 151170.80094988694
p_DA: [199.9999999999999, 0.0, 0.0, 101.36268623281629, 1.820810213759617e-13, 200.00000000000003, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, -8.526512829121202e-14, 200.0, 200.0, 0.0, 200.0, 0.0, 200.00000000000034, 0.0, 200.0]


In [27]:
W_new = n_scenarios - W
hours = 24
profits_out = []

# calculate the expected profit for the out of sample scenarios
exp_profit = 0
for i in 1:W_new
    profit_day = 0
    for hour in 1:hours
        profit = 0
        profit += unselected_scenarios[1][hour,"price"] * p_DA_values[hour] # should the day ahead price be fixed to the first scenario? So instead of i, 1?
        profit += unselected_scenarios[1][hour,"price"] * (unselected_scenarios[i][hour,"wind power"]-p_DA_values[hour])*((0.9.*unselected_scenarios[i][hour,"grid_excess"]) + 1.2.*(1-unselected_scenarios[i][hour,"grid_excess"]))
        profit_day += profit
    end
    push!(profits_out, profit_day)
    exp_profit += profit_day
end

exp_profit = exp_profit / W_new



30751.68994210677

In [28]:
# create a dataframe with the profits and if the scenanrio is in or out of sample
df = DataFrame(Scenario = 1:W + W_new, Profit = [profits_in; profits_out], In_Out = [fill("In",W); fill("Out",W_new)])  

# save the data
CSV.write("1_4_results/profits_one_price.csv", df)




"1_4_results/profits_one_price.csv"